# Week 10 - Advanced Data Crawling

## 1.Twitter 데이터 수집하기

트위터는 데이터를 수집할 수 있는 API를 제공한다. API는 크게 다음과 같은 두가지로 제공된다.
* REST API - 주어진 query에 맞는 데이터를 제공한다.
* Streaming API - 제시된 키워드 등이 포함된 데이터를 streaming 해준다.

트위터 API에 대응하는 여러 라이브러리가 있지만, 본 예제에서는 tweepy 를 사용한다. tweepy의 설치는 다음과 같다.
* pip install tweepy

트위터는 OAuth 인증을 통해 데이터에 접근이 가능하다. OAuth 인증을 하기 위해서는 트위터 개발자 사이트에 등록하고, 데이터 수집을 위한 앱을 등록해야 한다. 등록을 마치면 OAuth 인증을 위한 키가 제공되는데 해당 키를 이용하여 트위터 API를 사용할 수 있다.

(강의 슬라이드 참고)

### 1.1 tweepy 및 OAuth 설정

In [ ]:
import tweepy

# OAuth setup
consumer_key = ""
consumer_secret = ""
access_token = ""
access_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)

### 1.2 tweepy api 오브젝트 생성

In [ ]:
api = tweepy.API(auth)

In [ ]:
my_timeline = api.home_timeline()
my_timeline[0] # 만약 내 팔로워가 없다면 에러 발생(Null 값이므로)

### 1.3 특정 사용자의 타임라인 수집

In [ ]:
user = api.get_user(screen_name="Sonny7")
user

In [ ]:
user.id_str

In [ ]:
user = api.get_user(user_id=1496952010649833477)  # 만약 screen_name이 아닌 id를 알고 있다면
user

In [ ]:
user_timeline = api.user_timeline(screen_name="Sonny7")
tweets = []
for tweet in user_timeline:
    tweets.append(tweet)
tweets[0].text

In [ ]:
tweets[1].text

### 1.4 특정 유저의 친구 목록

In [ ]:
user = api.get_user(screen_name="Yunaaaa")
user.screen_name
user.followers_count

In [ ]:
for friend in user.friends():
    print(friend.screen_name)

(주의) `user.followers()`는 2022년 5월 현재 모두 508805 명이다. 트위터 API는 15분간 15개의 request call을 허용한다. 한번의 request 마다 20개의 followers를 가져온다면 모두 약 44381여개의 call이 필요하기 때문에 rate limit을 넘어서게 된다. 따라서 이 경우는 15개를 가져오고 잠시 쉬었다가 다시 가져오는 등의 방법을 사용한다. 처리 방법은 뒤에서 다루고 여기서는 call을 많이 사용하지 않는 friends의 리스트를 가져와 보았다. (API v1.0 기준)

API v2.0 에서는 rate limit 정책이 변화되었다. 다음의 링크를 참고하자.

https://developer.twitter.com/en/docs/twitter-api/rate-limits 

### 1.5 Pagination and Cursor

트위터의 정보는 pagination 되어 있다. page를 넘겨가며 데이터를 수집하기 위해서는 cursor 를 사용한다. Cursor는 다음과 같이 사용한다.

In [ ]:
my_tweets = []
for tweet in tweepy.Cursor(api.user_timeline, id="elonmusk").items(20):
    my_tweets.append(tweet)
    

for t in my_tweets:
    print(t.text + "\n\n--\n")

In [ ]:
my_tweets[0].text
# my_tweets[1].user.url

수집하고자 하는 트윗의 갯수를 지정할 수 있다. 이 경우 아이템의 숫자 (items()) 혹은 페이지의 숫자 (pages()) 를 지정한다.

```
# Only iterate through the first 200 statuses
for status in tweepy.Cursor(api.user_timeline).items(200):
    process_status(status)

# Only iterate through the first 3 pages
for page in tweepy.Cursor(api.user_timeline).pages(3):
    process_page(page)
```

### 1.6 Handling the rate limit using cursors

위에서 언급한 것처럼, 많은 양의 트윗을 수집하고자 할 때, **```RateLimitError```** 에러가 발생한다. 이런 경우 페이지를 넘기기 전에 (next()) 에러를 체크하고, **```RateLimitError```** 에러가 발생하면 그 만큼 수집을 멈추었다가 재개하여야 한다. 에러 핸들링을 하지 않으면 프로그램이 멈추고 데이터 수집도 멈춘다.

```
def limit_handled(cursor):
    while True:
        try:
            yield cursor.next()
        except tweepy.RateLimitError:
            time.sleep(15 * 60)

for follower in limit_handled(tweepy.Cursor(api.followers).items()):
    process_follower(follower)
```

## 2.Streaming Tweet Data

Streaming API는 rate limit의 제한이 없어 트윗을 수집할 때 유용하다. 그러나 모든 트윗을 제공하는 것은 아니고, 전체 트윗의 1%만을 random 으로 제공해 준다.

Streaming API를 사용하여 트윗을 수집하려면 다음과 같은 순서를 따른다.

* StreamingClient 클래스를 상속받은 MyListenerV2 클래스를 만든다.
* Stream 오브젝트를 생성한다.
* Stream 오브젝트에 Twitter API를 연결한다.

### 2.1 StreamListener 클래스 생성

In [ ]:
import tweepy
import time

class MyListenerV2(tweepy.StreamingClient): # tweepy의 기본 설정을 받아온다(상속한다)
    def on_connect(self): # 연결 성공 시 확인
        print("Connected.")
    
    def on_tweet(self, tweet):
        print(tweet.text)
        print("-"*100)
        time.sleep(1) # 속도를 조절하는 역할 (없어도 됨)

### 2.2 Stream 오브젝트 생성하고 Bearer Token 연결

In [ ]:
twitter_stream = MyListenerV2("") # bearer token을 대신 사용 : v2부터 auth(4개의 토큰 필요) 필요없이 작동한다

### 2.3. Stream 기본 규칙 지정
- 모든 트윗을 보는 것은 낭비가 심하기 때문에 몇가지 규칙을 정해서 필터링 양을 줄일 수 있다
- 세부적인 쿼리는 https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query 를 참조한다

In [ ]:
twitter_stream.add_rules(tweepy.StreamRule("(BTS) place_country:KR")) # BTS관련 Tweets 중, KR 소속인 트윗 보기

### 2.3. Streaming 시작

In [ ]:
twitter_stream.filter()

### 2.4 json 파일로 저장할 수 있도록 상속 클래스 수정

In [ ]:
# 결과 정렬해서 확인
from pprint import pprint
import json 

In [ ]:
import tweepy

class MyListenerV2(tweepy.StreamingClient): # tweepy의 기본 설정을 받아온다(상속한다)
    def on_connect(self): # 연결 성공 시 확인
        print("Connected.")
            
    def on_data(self, data):
        try:
            with open('tweet_stream.json','a',encoding='utf-8') as file: # raw 데이터는 binary 형태를 갖는다. 보기 편하게 바꾸자
                file.write(data.decode('utf-8'))
                file.write("\n")
                pprint(json.loads(data.decode('utf-8')))
                print("-"*100)
                return True
        except BaseException as e:
            print("Error on_data: {}".format(str(e)))
        return True    

In [ ]:
twitter_stream = MyListenerV2("") # bearer token을 대신 사용 : v2부터 auth(4개의 토큰 필요) 필요없이 작동한다
twitter_stream.add_rules(tweepy.StreamRule("(BTS) place_country:KR")) # BTS관련 Tweets 중, KR 속성 트윗 보기

In [ ]:
twitter_stream.filter()
#twitter_stream.filter(expansions="author_id", tweet_fields="created_at", user_fields="username")

(참고1) ```on_tweet vs. on_data```

```on_tweet```는 데이터를 tweepy object 형태로 return한다. 따라서 data.text와 같은 notation이 가능하다. 하지만 ```on_data```는 데이터를 str로 return 한다. 우리는 데이터를 .json 파일로 저장할 것이기 때문에 tweepy object보다는 str로 저장하여야 한다. 따라서 두번째 수정된 코드에서는 ```on_data```를 사용하였다.

(참고2) ```on_tweet vs. on_data```

두개의 메소드는 StreamingClient내부에 구현되어 있다. 클래스를 상속 받으면 부모 클래스에 구현된 메소드를 재정의(override)해서 사용할 수 있다.

### 2.5 터미널에서 프로그램 실행

jupyter notebook (Google Colab)이 편리하긴 하지만 어떤 경우에는 터미널에서 프로그램을 실행하여야 한다. 스트리밍한 데이터를 저장하는 프로그램은 계속 데이터를 받아오기 때문에 jupyter notebook에서 실행하면 메모리 로드가 많아 작동불능 상태에 빠지기 쉽다. 따라서 이 경우 터미널에서 프로그램을 실행시킨다.

* 첫번째 스트리밍 코드는 twitter-stream.py 로 저장되어 있다.
* 수정된 두번째 스트리밍 코드는 twitter-stream-save.py로 저장되어 있다.

실행은 터미널에서 다음과 같이 한다.

* python twitter-stream-save.py

프로그램의 종료는 CTRL-C 를 누른다

### 2.6 JSON 파일 불러오기

In [ ]:
import json
from pprint import pprint

json_data = []
with open("tweet_stream.json") as file:
    data = file.readlines()
    for d in data:
        json_data.append(json.loads(d))
        
len(json_data)

In [1]:
pip freeze

altair @ file:///tmp/build/80754af9/altair_1599835197802/workNote: you may need to restart the kernel to use updated packages.

argon2-cffi @ file:///home/conda/feedstock_root/build_artifacts/argon2-cffi_1640817743617/work
argon2-cffi-bindings @ file:///D:/bld/argon2-cffi-bindings_1649500515836/work
asttokens @ file:///home/conda/feedstock_root/build_artifacts/asttokens_1660605382950/work
async-generator==1.10
attrs @ file:///home/conda/feedstock_root/build_artifacts/attrs_1659291887007/work
backcall @ file:///home/conda/feedstock_root/build_artifacts/backcall_1592338393461/work
backports.functools-lru-cache @ file:///home/conda/feedstock_root/build_artifacts/backports.functools_lru_cache_1618230623929/work
beautifulsoup4 @ file:///home/conda/feedstock_root/build_artifacts/beautifulsoup4_1649463573192/work
bleach @ file:///home/conda/feedstock_root/build_artifacts/bleach_1656355450470/work
blinker==1.4
bokeh==2.4.3
brotlipy==0.7.0
cachetools @ file:///tmp/build/80754af9/cachetools_1619

In [ ]:
pprint(json_data[0])

In [ ]:
json_data[0]['data']

In [ ]:
json_data[0]['includes']

In [ ]:
json_data[0]['includes']['users'][0]['name']

In [ ]:
for tweet in json_data:
    if tweet['data']:
        print(tweet['data']['author_id'],"\t",tweet['data']['text'].replace("\n"," ").strip())